In [1]:
 !pip -q install yfinance

In [2]:
import bs4 as bs
import requests
import numpy as np
import pandas as pd
import yfinance as yf
import urllib
import datetime as dt

In [3]:
tickers = []
companies = []
req = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(req.text)
table = soup.find('table',{'class':'wikitable sortable'})
for row in table.findAll('tr')[1:]:
  info = row.findAll('td')
  try:
    #if info[3].text == 'Information Technology':
    tickers.append(info[0].text.replace('\n',''))
    full=info[1].text
    companies.append(full)
  except: pass

In [4]:

def future_dates(date,lb):
  dates=[]
  for i in range(lb):
    dates.append(date+dt.timedelta(days=i))
  return dates
def master(stock,dates,titles,look_back=3):
  i=0
  features =[]
  targets=[]
  names=[]
  while True:
    try:
      news = [titles[i],titles[i+1],titles[i+2]]
      news = ' '.join(news)
      today=dates[i]
      tomorrow = today+dt.timedelta(days=2)
      pct_change = stock.history(start=today,end=tomorrow)['Close'].pct_change()[-1]*100
      target=np.nan
      '''
      if pct_change>2:
        target = 2
      elif pct_change<-2:
        target = 0
      else:
        target = 1
      '''
      target = pct_change
      features.append(news)
      targets.append(target)
      names.append(stock)
    except Exception as e:
      if str(e) == 'list index out of range':
        break
    i+=1
  return features,targets,names

In [ ]:
months = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,
          'September':9,'October':10,'November':11,'December':12}
X,y=[],[]
names=[]
'''
for loop through every sector
add up past 2 days of every stock
rewrite code fuck me
'''
for ticker in tickers:
  print(ticker)
  req = requests.get(f'https://www.reuters.com/search/news?blob={ticker}', proxies=urllib.request.getproxies())
  soup = bs.BeautifulSoup(req.text,'lxml')
  div = soup.find('div',{'id':'quoteKeymatch'})
  stock = yf.Ticker(ticker)
  try:
    req2 = requests.get('https://'+div.find('a')['href'][2:], proxies=urllib.request.getproxies())
    soup2 = bs.BeautifulSoup(req2.text,'lxml')
    div = soup2.find('div',{'class':'Profile-feeds-jNLEN'})
    div = div.findAll('div',{'class':'item'})
    df = pd.DataFrame()
    dates=[]
    titles=[]
    blurbs = []
    targets=[]
    for article in div:
      a = article.find('a')
      p = article.find('p')
      url = a['href']
      title = a.text
      blurb = p.text
      title+=blurb
      req3 = requests.get(url)
      soup3 = bs.BeautifulSoup(req3.text,'lxml')
      div = soup3.find('div',{'class':'ArticleHeader_date'})
      try:
        date = div.text.split('/')[0]
        date = date.split(' ')
        month = months[date[0]]
        day = date[1][:-1]
        year = date[2]
        date = dt.date(int(year),int(month),int(day))
      
        dates.append(date)
        titles.append(title)
      except:pass
    xs,ys,ns=master(stock,dates,titles)
    X+=xs
    y+=ys
    names+=ns
    print(len(X))
  except Exception as e:
    print(e)


MMM
- MMM: No data found for this date range, symbol may be delisted
- MMM: No data found for this date range, symbol may be delisted
16
ABT
34
ABBV
- ABBV: No data found for this date range, symbol may be delisted
- ABBV: No data found for this date range, symbol may be delisted
50
ABMD
- ABMD: No data found for this date range, symbol may be delisted
67
ACN
85
ATVI
- ATVI: No data found for this date range, symbol may be delisted
102
ADBE
120
AMD
138
AAP
143
AES
161
AFL
179
A
197
APD
215
AKAM
233
ALK
251
ALB
269
ARE
282
ALXN
300
ALGN
318
ALLE
321
ADS
339
LNT
339
ALL
357
GOOGL
- GOOGL: No data found for this date range, symbol may be delisted
374
GOOG
- GOOG: No data found for this date range, symbol may be delisted
391
MO
409
AMZN
427
AMCR
'NoneType' object has no attribute 'findAll'
AEE
440
AAL
458
AEP
476
AXP
- AXP: No data found for this date range, symbol may be delisted
493
AIG
511
AMT
529
AWK
547
AMP
557
ABC
575
AME
581
AMGN
599
APH
607
ADI
623
ANSS
- ANSS: No data found for th

In [ ]:
df = pd.DataFrame()
df['stock']=names
df['text'] = X
df['target'] = y
df.to_csv('news_classed.csv')

In [ ]:
len(names)

7400